In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
import multiprocessing
from suite2p import extraction

In [1]:


BASE_DIR_1 = Path('F:/')
BASE_DIR_2 = Path('D:/')

mice = [25,27,30,36,39,52]
planes = range(1,9)

# Generate tuples of sessions
session_tuples = []
for mouse in mice:
    base_dir = BASE_DIR_1 if mouse < 31 else BASE_DIR_2
    for plane in planes:
        plane_dir = base_dir / f'{mouse:03}/plane_{plane}'
        sessions = [x.name for x in plane_dir.iterdir() if x.is_dir() and x.name[0].isdigit()]
        # list of (mouse,plane,session) tuples
        for session in sessions:
            session_dir = plane_dir / session / 'plane0'
            mask_fn = f'{mouse:03}_plane_{plane}_{session}_cp_masks.npy'

            if not (session_dir / mask_fn).exists():
                session_tuples.append((mouse,plane,session))
                print(f'{mouse:03}_plane_{plane}_{session} missing')

052_plane_2_001x missing
052_plane_2_006x missing
052_plane_4_029x missing


In [7]:
len(session_tuples)

47

In [12]:
meanimge_exists = []
for mouse, plane, session in session_tuples:
    base_dir = BASE_DIR_1 if mouse < 31 else BASE_DIR_2
    session_dir = base_dir / f'{mouse:03}/plane_{plane}/{session}/plane0'
    ops_fn = session_dir / 'ops.npy'
    ops = np.load(ops_fn, allow_pickle=True).item()
    if not 'meanImgE' in ops.keys():
        ops = extraction.enhanced_mean_image(ops)
        np.save(ops_fn, ops)

added enhanced mean image
added enhanced mean image
added enhanced mean image
added enhanced mean image
added enhanced mean image
added enhanced mean image


In [11]:
len(np.where(meanimge_exists)[0])

41

# Check if merging is done
- by 230910_roi_collation_parallel.py

In [87]:
mice = [25,27,30,36,39,52]
planes = range(1,9)
base_dir = Path(r'E:\TPM\JK\h5')

# Generate tuples of sessions
session_tuples_missing_final = []
session_tuples_missing_merged = []
for mouse in mice:
    for plane in planes:
        plane_dir = base_dir / f'{mouse:03}/plane_{plane}'
        sessions = [x.name for x in plane_dir.iterdir() if x.is_dir() and x.name[0].isdigit() and x.name[-1].isdigit()]
        # list of (mouse,plane,session) tuples
        for session in sessions:
            session_dir = plane_dir / session / 'plane0' / 'roi'
            final_merge_fn = f'final_roi_results_{mouse:03}_plane_{plane}_{session}_wo_dendrite_filtering.npy'
            merge_fn = f'merged_masks_{mouse:03}_plane_{plane}_{session}.npy'

            if not (session_dir / merge_fn).exists():
                session_tuples_missing_merged.append((mouse,plane,session))
            elif not (session_dir / final_merge_fn).exists():
                session_tuples_missing_final.append((mouse,plane,session))


In [88]:
len(session_tuples_missing_final)

16

In [89]:
len(session_tuples_missing_merged)

16

In [90]:
session_tuples_missing_final

[(25, 1, '5555_002'),
 (25, 8, '002'),
 (27, 1, '006'),
 (27, 4, '025'),
 (27, 6, '5554_002'),
 (30, 4, '9999_2'),
 (36, 6, '9998_2'),
 (39, 8, '5554_110'),
 (52, 2, '022'),
 (52, 5, '001'),
 (52, 5, '020'),
 (52, 5, '028'),
 (52, 6, '011'),
 (52, 6, '013'),
 (52, 6, '019'),
 (52, 6, '029')]

In [7]:
# All these merge error sessions have masks created

for tuple in session_tuples:
    mouse = tuple[0]
    plane = tuple[1]
    session = tuple[2]
    session_dir = base_dir / f'{mouse:03}/plane_{plane}/{session}/plane0/'
    mask_fn = f'{mouse:03}_plane_{plane}_{session}_cp_masks.npy'
    if not (session_dir / mask_fn).exists():
        print(f'{mouse:03}_plane_{plane}_{session} done')


# Fix problems
- 2 modes of failure
    - No merge at all
         - Problem with 0 unique mask. Fixed in def clean_and_merge_masks
    - Final merge problem
          - multiple mask2 matched rois in <deal_with_one_to_one>
          - These are neighboring ROIs seemingly single ROI, so just compare with the sum area